In [3]:
import os
from functools import partial
import torch
import numpy as np
import pandas as pd

import pyro
import pyro.distributions as dist

In [4]:
# for Continuous Integration testing
smoke_test = ('CI' in os.environ)
# assert pyro.__version__.startswith('1.3.0')
pyro.enable_validation(True)
pyro.set_rng_seed(1)
pyro.enable_validation(True)

In [56]:
 mapk_data=pd.read_csv("data\\mapk100_rm1.csv")

In [57]:
mapk_data.head()

,Unnamed: 0,Raf,Mek,Erk
0,2,47,64,82
1,3,47,62,84
2,4,50,66,89
3,5,44,66,91
4,6,42,55,82


In [58]:
mapk_data=mapk_data.drop(columns='Unnamed: 0')

In [59]:
from torch import nn
from pyro.nn import PyroModule

assert issubclass(PyroModule[nn.Linear], nn.Linear)
assert issubclass(PyroModule[nn.Linear], PyroModule)

In [60]:
data=data = torch.tensor(mapk_data[["Raf", "Mek", "Erk"]].values,
                        dtype=torch.float)

In [64]:
x_data, y_data = data[:,1:2], data[:,2]

x_squared_data = torch.cat((x_data, x_data*x_data, 1/x_data), 1)

tensor([82., 84., 89., 91., 82.])

In [65]:
# Regression model
linear_reg_model = PyroModule[nn.Linear](3,1)

# Define loss and optimize
loss_fn = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.Adam(linear_reg_model.parameters(), lr=0.01)
num_iterations = 10000 if not smoke_test else 2

def train():
    # run the model forward on the data
    y_pred = linear_reg_model(x_squared_data).squeeze(-1)
    # calculate the mse loss
    loss = loss_fn(y_pred, y_data)
    # initialize gradients to zero
    optim.zero_grad()
    # backpropagate
    loss.backward()
    # take a gradient step
    optim.step()
    return loss

for j in range(num_iterations):
    loss = train()
    if (j + 1) % 50 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))

# Inspect learned parameters
print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

[iteration 0050] loss: 187905.7344
[iteration 0100] loss: 18869.7480
[iteration 0150] loss: 13182.9131
[iteration 0200] loss: 13151.2852
[iteration 0250] loss: 13128.2686
[iteration 0300] loss: 13102.7725
[iteration 0350] loss: 13074.7549
[iteration 0400] loss: 13044.3252
[iteration 0450] loss: 13011.5859
[iteration 0500] loss: 12976.6055
[iteration 0550] loss: 12939.4463
[iteration 0600] loss: 12900.1553
[iteration 0650] loss: 12858.7754
[iteration 0700] loss: 12815.3340
[iteration 0750] loss: 12769.8623
[iteration 0800] loss: 12722.3838
[iteration 0850] loss: 12672.9131
[iteration 0900] loss: 12621.4766
[iteration 0950] loss: 12568.0801
[iteration 1000] loss: 12512.7402
[iteration 1050] loss: 12455.4688
[iteration 1100] loss: 12396.2725
[iteration 1150] loss: 12335.1631
[iteration 1200] loss: 12272.1455
[iteration 1250] loss: 12207.2236
[iteration 1300] loss: 12140.4150
[iteration 1350] loss: 12071.7148
[iteration 1400] loss: 12001.1367
[iteration 1450] loss: 11928.6777
[iteration 15

In [39]:
print(mapk_data[:1])
linear_reg_model(torch.tensor([46.8, 46.8*46.8]))

   Raf  Mek  Erk
0   47   64   82


tensor([61.7021], grad_fn=<AddBackward0>)

##### Bayesian Regression with Pyro’s Stochastic Variational Inference (SVI)

In [42]:
from pyro.nn import PyroSample


class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean

In [44]:
from pyro.infer.autoguide import AutoDiagonalNormal

model = BayesianRegression(1, 1)
guide = AutoDiagonalNormal(model)

In [45]:
from pyro.infer import SVI, Trace_ELBO


adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [46]:
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

[iteration 0001] loss: 190.6467
[iteration 0101] loss: 3.4870
[iteration 0201] loss: 3.2757
[iteration 0301] loss: 3.7104
[iteration 0401] loss: 3.2371
[iteration 0501] loss: 3.2677
[iteration 0601] loss: 3.2544
[iteration 0701] loss: 3.2334
[iteration 0801] loss: 3.3157
[iteration 0901] loss: 3.2346
[iteration 1001] loss: 3.2213
[iteration 1101] loss: 3.2331
[iteration 1201] loss: 3.3399
[iteration 1301] loss: 3.2730
[iteration 1401] loss: 3.2356


In [47]:
guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

AutoDiagonalNormal.loc Parameter containing:
tensor([0.4395, 1.2235, 5.9731])
AutoDiagonalNormal.scale tensor([0.1073, 0.0233, 0.7513])


In [48]:
guide.quantiles([0.25, 0.5, 0.75])

{'sigma': [tensor(5.9076), tensor(6.0814), tensor(6.2524)],
 'linear.weight': [tensor([[1.2078]]), tensor([[1.2235]]), tensor([[1.2392]])],
 'linear.bias': [tensor([5.4663]), tensor([5.9731]), tensor([6.4798])]}